# Comparing Before Budgeted CPU versus After Budgeted CPU

### This creates the raw "Red and Green" sheet master data

### Overall Process

- Get current or "After" budgeted CPU file
- Get "Before" budgeted CPU file
- Merge between the 2 using full OUTER JOIN
- Create 2 new columns:
    1. After Budgeted CPU minus Before Budgeted CPU (at group-subgroup level)
    2. After Budgeted CPU minus Before Budgeted CPU (at GraphCat level)
- Export to Excel and save in ```\\207.130.185.67\aqgbudget2\Cost\Reserve Adjustments\Reports\Normal Reserve Balance Verification\RVMS_Before_After_Checks``` folder

In [1]:
from datetime import datetime
from dateutil import relativedelta
from pathlib import Path
import os
import pandas as pd
import pyodbc
import scrapbook as sb
import time
from win10toast import ToastNotifier
pd.options.display.max_rows=1000
pd.options.display.max_columns=100

### Enter CLA Claim Month:

In [ ]:
# If using papermill, have to comment this out.  It doesn't support getting input from the user
# CLA_MONTH = input("Enter CLA Claim Month ('YYYYMM'): ")

In [2]:
CLA_MONTH = '201903'

### Define where to save the Red and Green raw data file based on CLA claim month:

In [3]:
base_dir = "//207.130.185.67/aqgbudget2/Cost/Reserve Adjustments/Reports/Normal Reserve Balance Verification/RVMS_Before_After_Checks"
p = Path(base_dir)
save_dir = p / CLA_MONTH
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

### Define where to retrieve the current budgeted CPUs ("after" CPUs):

In [4]:
current_cpu_dir = Path("//207.130.185.67/aqgbudget2/Cost/Reserve Adjustments/Reports/Normal Reserve Balance Verification/RVMS_Current_Budgeted_CPUs/" 
                       + CLA_MONTH + "/All_Plants_Budgeted_CPU_By_Group_SubGroup.xlsx")

#### Now fetch the "after" CPUs:

In [5]:
cpu_after = pd.read_excel(current_cpu_dir)

In [6]:
cpu_after.shape

(59160, 14)

In [7]:
cpu_after['Group-SubGroup'] = cpu_after['GRP_NM'].map(str) + ' - ' + cpu_after['SUBGRP_NM'].map(str)

In [8]:
cpu_after.head()

,GraphCatID,GraphCatDesc,GRP_NM,SUBGRP_NM,Budgeted_CPU_SubGroup_Level,Planned_Sales_RVMS,Budgeted_CPU_GC_Level,Orig_Saturation_CPU_GC_Level,Cum_Actual_CPU_GC_Level,RVMS_Claim_Month,ModelYear,Factory,ModelName,DestCode,Group-SubGroup
0,449,R HMI CIVIC 2010 4DR KA,Chassis,Calipers/Disk/Drums/Pads,0.834396,94558,102.01,152.72,101.7,201903,2010,HMI,CIVIC,KA,Chassis - Calipers/Disk/Drums/Pads
1,449,R HMI CIVIC 2010 4DR KA,Chassis,Drive Shaft,15.891899,94558,102.01,152.72,101.7,201903,2010,HMI,CIVIC,KA,Chassis - Drive Shaft
2,449,R HMI CIVIC 2010 4DR KA,Chassis,Hand/cable Brakes,0.048017,94558,102.01,152.72,101.7,201903,2010,HMI,CIVIC,KA,Chassis - Hand/cable Brakes
3,449,R HMI CIVIC 2010 4DR KA,Chassis,Master Cylinder,0.049997,94558,102.01,152.72,101.7,201903,2010,HMI,CIVIC,KA,Chassis - Master Cylinder
4,449,R HMI CIVIC 2010 4DR KA,Chassis,Steering Vibration,0.325561,94558,102.01,152.72,101.7,201903,2010,HMI,CIVIC,KA,Chassis - Steering Vibration


In [9]:
after_column_names = [col + '_After' for col in cpu_after.columns]

In [10]:
after_column_names

['GraphCatID_After',
 'GraphCatDesc_After',
 'GRP_NM_After',
 'SUBGRP_NM_After',
 'Budgeted_CPU_SubGroup_Level_After',
 'Planned_Sales_RVMS_After',
 'Budgeted_CPU_GC_Level_After',
 'Orig_Saturation_CPU_GC_Level_After',
 'Cum_Actual_CPU_GC_Level_After',
 'RVMS_Claim_Month_After',
 'ModelYear_After',
 'Factory_After',
 'ModelName_After',
 'DestCode_After',
 'Group-SubGroup_After']

In [11]:
cpu_after.columns = after_column_names

In [12]:
cpu_after.head()

,GraphCatID_After,GraphCatDesc_After,GRP_NM_After,SUBGRP_NM_After,Budgeted_CPU_SubGroup_Level_After,Planned_Sales_RVMS_After,Budgeted_CPU_GC_Level_After,Orig_Saturation_CPU_GC_Level_After,Cum_Actual_CPU_GC_Level_After,RVMS_Claim_Month_After,ModelYear_After,Factory_After,ModelName_After,DestCode_After,Group-SubGroup_After
0,449,R HMI CIVIC 2010 4DR KA,Chassis,Calipers/Disk/Drums/Pads,0.834396,94558,102.01,152.72,101.7,201903,2010,HMI,CIVIC,KA,Chassis - Calipers/Disk/Drums/Pads
1,449,R HMI CIVIC 2010 4DR KA,Chassis,Drive Shaft,15.891899,94558,102.01,152.72,101.7,201903,2010,HMI,CIVIC,KA,Chassis - Drive Shaft
2,449,R HMI CIVIC 2010 4DR KA,Chassis,Hand/cable Brakes,0.048017,94558,102.01,152.72,101.7,201903,2010,HMI,CIVIC,KA,Chassis - Hand/cable Brakes
3,449,R HMI CIVIC 2010 4DR KA,Chassis,Master Cylinder,0.049997,94558,102.01,152.72,101.7,201903,2010,HMI,CIVIC,KA,Chassis - Master Cylinder
4,449,R HMI CIVIC 2010 4DR KA,Chassis,Steering Vibration,0.325561,94558,102.01,152.72,101.7,201903,2010,HMI,CIVIC,KA,Chassis - Steering Vibration


#### Get "Before" budgeted CPU file and rename columns:

In [13]:
conn_str = (
            r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
            r'DBQ=\\207.130.185.67\aqgbudget2\Cost\Reserve Adjustments\Reports\databases\RVMS.accdb;'
           )
cnxn = pyodbc.connect(conn_str)
cursor = cnxn.cursor()

sql = """
SELECT
    *
    
FROM tbl_Historical_Budgeted_CPU

WHERE
    RVMS_Claim_Month = (SELECT max(tbl_Historical_Budgeted_CPU.[RVMS_Claim_Month]) FROM tbl_Historical_Budgeted_CPU)
"""

try:
    cpu_before = pd.read_sql(sql, cnxn)
    #cpu_before = pd.read_sql(sql, cnxn)
    
    # Close connections
    cursor.close()
    cnxn.close()
except:
    print("Error connecting to database")
    cursor.close()
    cnxn.close()

In [14]:
cpu_before.shape

(59160, 15)

In [15]:
cpu_before.head()

,ID,GraphCatID,GraphCatDesc,GRP_NM,SUBGRP_NM,Budgeted_CPU_SubGroup_Level,Planned_Sales_RVMS,Budgeted_CPU_GC_Level,Orig_Saturation_CPU_GC_Level,Cum_Actual_CPU_GC_Level,RVMS_Claim_Month,ModelYear,Factory,ModelName,DestCode
0,222634,449,R HMI CIVIC 2010 4DR KA,Chassis,Calipers/Disk/Drums/Pads,0.834396,94558,102.01,152.72,101.69,201903,2010,HMI,CIVIC,KA
1,222635,449,R HMI CIVIC 2010 4DR KA,Chassis,Drive Shaft,15.891899,94558,102.01,152.72,101.69,201903,2010,HMI,CIVIC,KA
2,222636,449,R HMI CIVIC 2010 4DR KA,Chassis,Hand/cable Brakes,0.048017,94558,102.01,152.72,101.69,201903,2010,HMI,CIVIC,KA
3,222637,449,R HMI CIVIC 2010 4DR KA,Chassis,Master Cylinder,0.049997,94558,102.01,152.72,101.69,201903,2010,HMI,CIVIC,KA
4,222638,449,R HMI CIVIC 2010 4DR KA,Chassis,Steering Vibration,0.325561,94558,102.01,152.72,101.69,201903,2010,HMI,CIVIC,KA


In [16]:
before_column_names = [col + '_Before' for col in cpu_before.columns]

In [17]:
cpu_before.columns = before_column_names

In [18]:
cpu_before.head()

,ID_Before,GraphCatID_Before,GraphCatDesc_Before,GRP_NM_Before,SUBGRP_NM_Before,Budgeted_CPU_SubGroup_Level_Before,Planned_Sales_RVMS_Before,Budgeted_CPU_GC_Level_Before,Orig_Saturation_CPU_GC_Level_Before,Cum_Actual_CPU_GC_Level_Before,RVMS_Claim_Month_Before,ModelYear_Before,Factory_Before,ModelName_Before,DestCode_Before
0,222634,449,R HMI CIVIC 2010 4DR KA,Chassis,Calipers/Disk/Drums/Pads,0.834396,94558,102.01,152.72,101.69,201903,2010,HMI,CIVIC,KA
1,222635,449,R HMI CIVIC 2010 4DR KA,Chassis,Drive Shaft,15.891899,94558,102.01,152.72,101.69,201903,2010,HMI,CIVIC,KA
2,222636,449,R HMI CIVIC 2010 4DR KA,Chassis,Hand/cable Brakes,0.048017,94558,102.01,152.72,101.69,201903,2010,HMI,CIVIC,KA
3,222637,449,R HMI CIVIC 2010 4DR KA,Chassis,Master Cylinder,0.049997,94558,102.01,152.72,101.69,201903,2010,HMI,CIVIC,KA
4,222638,449,R HMI CIVIC 2010 4DR KA,Chassis,Steering Vibration,0.325561,94558,102.01,152.72,101.69,201903,2010,HMI,CIVIC,KA


In [19]:
try:
    assert cpu_before.shape[0] == cpu_after.shape[0]
except:
    toaster = ToastNotifier()
    toaster.show_toast("### ERROR ###",
                   "Number of rows don't match between CPU after and CPU before data sets",
                   icon_path=None,
                   duration=5)
    print('ERROR!!! - Number of rows do not match between CPU after and CPU before data sets')

### Merge the after CPU data set with the before CPU data set

In [20]:
cpu_before_after_merge = pd.merge(cpu_after, cpu_before, how='outer', 
                                  left_on=['GraphCatID_After','GRP_NM_After','SUBGRP_NM_After'], 
                                  right_on=['GraphCatID_Before','GRP_NM_Before','SUBGRP_NM_Before']
                                 )

In [21]:
cpu_before_after_merge.shape

(59160, 30)

In [22]:
cpu_before_after_merge.head()

,GraphCatID_After,GraphCatDesc_After,GRP_NM_After,SUBGRP_NM_After,Budgeted_CPU_SubGroup_Level_After,Planned_Sales_RVMS_After,Budgeted_CPU_GC_Level_After,Orig_Saturation_CPU_GC_Level_After,Cum_Actual_CPU_GC_Level_After,RVMS_Claim_Month_After,ModelYear_After,Factory_After,ModelName_After,DestCode_After,Group-SubGroup_After,ID_Before,GraphCatID_Before,GraphCatDesc_Before,GRP_NM_Before,SUBGRP_NM_Before,Budgeted_CPU_SubGroup_Level_Before,Planned_Sales_RVMS_Before,Budgeted_CPU_GC_Level_Before,Orig_Saturation_CPU_GC_Level_Before,Cum_Actual_CPU_GC_Level_Before,RVMS_Claim_Month_Before,ModelYear_Before,Factory_Before,ModelName_Before,DestCode_Before
0,449,R HMI CIVIC 2010 4DR KA,Chassis,Calipers/Disk/Drums/Pads,0.834396,94558,102.01,152.72,101.7,201903,2010,HMI,CIVIC,KA,Chassis - Calipers/Disk/Drums/Pads,222634,449,R HMI CIVIC 2010 4DR KA,Chassis,Calipers/Disk/Drums/Pads,0.834396,94558,102.01,152.72,101.69,201903,2010,HMI,CIVIC,KA
1,449,R HMI CIVIC 2010 4DR KA,Chassis,Drive Shaft,15.891899,94558,102.01,152.72,101.7,201903,2010,HMI,CIVIC,KA,Chassis - Drive Shaft,222635,449,R HMI CIVIC 2010 4DR KA,Chassis,Drive Shaft,15.891899,94558,102.01,152.72,101.69,201903,2010,HMI,CIVIC,KA
2,449,R HMI CIVIC 2010 4DR KA,Chassis,Hand/cable Brakes,0.048017,94558,102.01,152.72,101.7,201903,2010,HMI,CIVIC,KA,Chassis - Hand/cable Brakes,222636,449,R HMI CIVIC 2010 4DR KA,Chassis,Hand/cable Brakes,0.048017,94558,102.01,152.72,101.69,201903,2010,HMI,CIVIC,KA
3,449,R HMI CIVIC 2010 4DR KA,Chassis,Master Cylinder,0.049997,94558,102.01,152.72,101.7,201903,2010,HMI,CIVIC,KA,Chassis - Master Cylinder,222637,449,R HMI CIVIC 2010 4DR KA,Chassis,Master Cylinder,0.049997,94558,102.01,152.72,101.69,201903,2010,HMI,CIVIC,KA
4,449,R HMI CIVIC 2010 4DR KA,Chassis,Steering Vibration,0.325561,94558,102.01,152.72,101.7,201903,2010,HMI,CIVIC,KA,Chassis - Steering Vibration,222638,449,R HMI CIVIC 2010 4DR KA,Chassis,Steering Vibration,0.325561,94558,102.01,152.72,101.69,201903,2010,HMI,CIVIC,KA


### Create columns that represent the before and after CPUs at GraphCat level, subgroup level, and total adjustment costs

In [23]:
cpu_before_after_merge['Budgeted_CPU_SubGroup_Level_After_Minus_Before'] = cpu_before_after_merge['Budgeted_CPU_SubGroup_Level_After'] \
                                                            - cpu_before_after_merge['Budgeted_CPU_SubGroup_Level_Before']
cpu_before_after_merge['Budgeted_CPU_GC_Level_After_Minus_Before'] = cpu_before_after_merge['Budgeted_CPU_GC_Level_After'] \
                                                                     - cpu_before_after_merge['Budgeted_CPU_GC_Level_Before']
cpu_before_after_merge['CPU_DIFF_SubGroup_Level_x_SALES'] = cpu_before_after_merge['Budgeted_CPU_SubGroup_Level_After_Minus_Before'] \
                                                      * cpu_before_after_merge['Planned_Sales_RVMS_After']
cpu_before_after_merge['CPU_DIFF_GC_LEVEL_x_SALES'] = cpu_before_after_merge['Budgeted_CPU_GC_Level_After_Minus_Before'] \
                                                      * cpu_before_after_merge['Planned_Sales_RVMS_After']

In [24]:
cpu_before_after_merge.head()

,GraphCatID_After,GraphCatDesc_After,GRP_NM_After,SUBGRP_NM_After,Budgeted_CPU_SubGroup_Level_After,Planned_Sales_RVMS_After,Budgeted_CPU_GC_Level_After,Orig_Saturation_CPU_GC_Level_After,Cum_Actual_CPU_GC_Level_After,RVMS_Claim_Month_After,ModelYear_After,Factory_After,ModelName_After,DestCode_After,Group-SubGroup_After,ID_Before,GraphCatID_Before,GraphCatDesc_Before,GRP_NM_Before,SUBGRP_NM_Before,Budgeted_CPU_SubGroup_Level_Before,Planned_Sales_RVMS_Before,Budgeted_CPU_GC_Level_Before,Orig_Saturation_CPU_GC_Level_Before,Cum_Actual_CPU_GC_Level_Before,RVMS_Claim_Month_Before,ModelYear_Before,Factory_Before,ModelName_Before,DestCode_Before,Budgeted_CPU_SubGroup_Level_After_Minus_Before,Budgeted_CPU_GC_Level_After_Minus_Before,CPU_DIFF_SubGroup_Level_x_SALES,CPU_DIFF_GC_LEVEL_x_SALES
0,449,R HMI CIVIC 2010 4DR KA,Chassis,Calipers/Disk/Drums/Pads,0.834396,94558,102.01,152.72,101.7,201903,2010,HMI,CIVIC,KA,Chassis - Calipers/Disk/Drums/Pads,222634,449,R HMI CIVIC 2010 4DR KA,Chassis,Calipers/Disk/Drums/Pads,0.834396,94558,102.01,152.72,101.69,201903,2010,HMI,CIVIC,KA,0.0,0.0,0.0,0.0
1,449,R HMI CIVIC 2010 4DR KA,Chassis,Drive Shaft,15.891899,94558,102.01,152.72,101.7,201903,2010,HMI,CIVIC,KA,Chassis - Drive Shaft,222635,449,R HMI CIVIC 2010 4DR KA,Chassis,Drive Shaft,15.891899,94558,102.01,152.72,101.69,201903,2010,HMI,CIVIC,KA,0.0,0.0,0.0,0.0
2,449,R HMI CIVIC 2010 4DR KA,Chassis,Hand/cable Brakes,0.048017,94558,102.01,152.72,101.7,201903,2010,HMI,CIVIC,KA,Chassis - Hand/cable Brakes,222636,449,R HMI CIVIC 2010 4DR KA,Chassis,Hand/cable Brakes,0.048017,94558,102.01,152.72,101.69,201903,2010,HMI,CIVIC,KA,0.0,0.0,0.0,0.0
3,449,R HMI CIVIC 2010 4DR KA,Chassis,Master Cylinder,0.049997,94558,102.01,152.72,101.7,201903,2010,HMI,CIVIC,KA,Chassis - Master Cylinder,222637,449,R HMI CIVIC 2010 4DR KA,Chassis,Master Cylinder,0.049997,94558,102.01,152.72,101.69,201903,2010,HMI,CIVIC,KA,0.0,0.0,0.0,0.0
4,449,R HMI CIVIC 2010 4DR KA,Chassis,Steering Vibration,0.325561,94558,102.01,152.72,101.7,201903,2010,HMI,CIVIC,KA,Chassis - Steering Vibration,222638,449,R HMI CIVIC 2010 4DR KA,Chassis,Steering Vibration,0.325561,94558,102.01,152.72,101.69,201903,2010,HMI,CIVIC,KA,0.0,0.0,0.0,0.0


### Define file name format:

In [25]:
date_hour_stamp = time.strftime('%Y-%m-%d_%H_%M')
file_name = 'All_Plants_Before_After_Budgeted_CPUs_' + date_hour_stamp + '.xlsx'

### Write/save file to designated network share drive location:

In [26]:
cpu_before_after_merge.to_excel(save_dir / file_name, index=False)

### Now, we need to "glue" the location of the saved file location to this notebook so that another notebook can retrieve/reference from it:

In [27]:
str(save_dir / file_name)

'\\\\207.130.185.67\\aqgbudget2\\Cost\\Reserve Adjustments\\Reports\\Normal Reserve Balance Verification\\RVMS_Before_After_Checks\\201903\\All_Plants_Before_After_Budgeted_CPUs_2019-04-25_14_17.xlsx'

In [28]:
sb.glue("path_to_red_green_sheet_excel_file", str(save_dir / file_name))

### Send Windows Toast notification when script completes

In [29]:
toaster = ToastNotifier()
toaster.show_toast("### Before vs After CPU Status ###",
                   "Successfuly compared before CPUs with after CPU adjustments",
                   icon_path="images/honda_logo.ico",
                   duration=5)

True